# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [2]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "/Users/imanjean-jacques/Documents/Data_Analytics_Bootcamp/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Burns Lake,CA,54.2331,-125.7533,22.82,77,100,3.74
1,1,Hithadhoo,MV,-0.6000,73.0833,81.21,75,100,15.39
2,2,Henties Bay,NaN,-22.1160,14.2845,56.93,87,31,5.50
3,3,Upernavik,GL,72.7868,-56.1549,14.85,50,30,4.59
4,4,Tamiahua,MX,21.2667,-97.4500,74.34,89,10,2.24


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [3]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 80


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [4]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]  
# Display sample data
preferred_cities_df.head() 

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
6,6,Kapaa,US,22.0752,-159.3190,79.12,86,75,12.66
7,7,Acarau,BR,-2.8856,-40.1200,77.00,85,32,7.94
11,11,Bambous Virieux,MU,-20.3428,57.7575,78.22,69,40,12.66
22,22,Areia Branca,BR,-4.9561,-37.1369,76.08,79,59,14.29
24,24,Faanui,PF,-16.4833,-151.7500,77.54,80,100,15.90


In [5]:
preferred_cities_df.isnull()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
6,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False
11,False,False,False,False,False,False,False,False,False
22,False,False,False,False,False,False,False,False,False
24,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
636,False,False,False,False,False,False,False,False,False
653,False,False,False,False,False,False,False,False,False
654,False,False,False,False,False,False,False,False,False
705,False,False,False,False,False,False,False,False,False


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [6]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
6,6,Kapaa,US,22.0752,-159.3190,79.12,86,75,12.66
7,7,Acarau,BR,-2.8856,-40.1200,77.00,85,32,7.94
11,11,Bambous Virieux,MU,-20.3428,57.7575,78.22,69,40,12.66
22,22,Areia Branca,BR,-4.9561,-37.1369,76.08,79,59,14.29
24,24,Faanui,PF,-16.4833,-151.7500,77.54,80,100,15.90


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [7]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# Display sample data
hotel_df.head()

,City,Country,Max Temp,Lat,Lng
6,Kapaa,US,79.12,22.0752,-159.3190
7,Acarau,BR,77.00,-2.8856,-40.1200
11,Bambous Virieux,MU,78.22,-20.3428,57.7575
22,Areia Branca,BR,76.08,-4.9561,-37.1369
24,Faanui,PF,77.54,-16.4833,-151.7500


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [8]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
6,Kapaa,US,79.12,22.0752,-159.3190,
7,Acarau,BR,77.00,-2.8856,-40.1200,
11,Bambous Virieux,MU,78.22,-20.3428,57.7575,
22,Areia Branca,BR,76.08,-4.9561,-37.1369,
24,Faanui,PF,77.54,-16.4833,-151.7500,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [9]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [10]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [11]:
# Display sample data
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
6,Kapaa,US,79.12,22.0752,-159.3190,
7,Acarau,BR,77.00,-2.8856,-40.1200,
11,Bambous Virieux,MU,78.22,-20.3428,57.7575,
22,Areia Branca,BR,76.08,-4.9561,-37.1369,
24,Faanui,PF,77.54,-16.4833,-151.7500,


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [24]:
# Drop the rows where there is no Hotel Name.
#clean_hotel_df = hotel_df.dropna(["Hotel Name"], axis=1)

# Display sample data
#clean_hotel_df.head()

**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [17]:
# In case of an empty DataFrame, load the sample data provided *Unable to download backup data. Gets error message
clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup_D2D3.csv")
clean_hotel_df.head(10)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,2,Aksu,CN,82.42,clear sky,41.12,80.26,Pudong Holiday Hotel
1,7,Touros,BR,80.60,broken clouds,-5.20,-35.46,INN NEW HORIZON
2,10,Morehead,US,75.20,clear sky,37.27,-87.18,CCI Express Inn
3,11,Port Elizabeth,ZA,84.20,clear sky,-33.92,25.57,39 On Nile Guest House
4,16,Northam,GB,82.40,few clouds,51.03,-4.22,Durrant House Hotel
5,18,Hyderabad,IN,87.01,haze,17.38,78.47,"Taj Krishna, Hyderabad"
6,20,Port Alfred,ZA,82.00,clear sky,-33.59,26.89,The Halyards Hotel
7,22,Atuona,PF,79.72,clear sky,-9.80,-139.03,Villa Enata
8,23,Kapaa,US,73.40,heavy intensity rain,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
9,25,Nikolskoye,RU,88.00,clear sky,59.70,30.79,Tourist House - Sablino


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [18]:
# Set the file name.
output_data_file = "WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file, index_label='Hotel_ID')

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [19]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [20]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Pudong Holiday Hotel</dd>\n<dt>City</dt><dd>Aksu</dd>\n<dt>Country</dt><dd>CN</dd>\n<dt>Current Weather</dt><dd>clear sky and 82.42 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>INN NEW HORIZON</dd>\n<dt>City</dt><dd>Touros</dd>\n<dt>Country</dt><dd>BR</dd>\n<dt>Current Weather</dt><dd>broken clouds and 80.6 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>CCI Express Inn</dd>\n<dt>City</dt><dd>Morehead</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Weather</dt><dd>clear sky and 75.2 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>39 On Nile Guest House</dd>\n<dt>City</dt><dd>Port Elizabeth</dd>\n<dt>Country</dt><dd>ZA</dd>\n<dt>Current Weather</dt><dd>clear sky and 84.2 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Durrant House Hotel</dd>\n<dt>City</dt><dd>Northam</dd>\n<dt>Country</dt><dd>GB</dd>\n<dt>Current Weather</dt><dd>few clouds and 82.4 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Taj Krishna, Hyderabad</dd>\n<dt>Cit

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [21]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
0,41.12,80.26
1,-5.20,-35.46
2,37.27,-87.18
3,-33.92,25.57
4,51.03,-4.22
5,17.38,78.47
6,-33.59,26.89
7,-9.80,-139.03
8,22.08,-159.32
9,59.70,30.79


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [23]:
# Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)


# Create a figure to add the Google map as a layer
fig = gmaps.figure()
fig.add_layer(marker_layer)

# Display the figure containing the map
fig

Figure(layout=FigureLayout(height='420px'))